In [1]:
#Bin5_steele_multiplier
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
import datetime
from pandas import ExcelWriter
#from pandas import Excelfile

In [2]:
path = Path('W:') / 'Amber Ag' / 'Amber clean data/Bin 13 review'

In [3]:
xls = pd.ExcelFile ('W:/Amber Ag/Amber clean data/Bin 13 review/Insert_bin13_stage3.xlsx')
df = pd.read_excel(xls, sheet_name = 'Bin133')
df

,Unnamed: 0,Steady date,OPI_DATE,Avg_Temp,RH,Avg_MC
0,1,2020-11-03 14:51:00,2020-11-03 14:51:00.000,24,55.64,21
1,2,2020-11-03 15:51:00,2020-11-03 15:51:00.000,24,56.38,16
2,3,2020-11-03 16:51:00,2020-11-03 16:51:00.000,30,59.72,21
3,4,2020-11-03 17:51:00,2020-11-04 16:50:59.995,30,59.72,16


In [4]:
df['Mt'] = np.nan

In [6]:
for index, row in df.iterrows():
    temp = row['Avg_Temp']
    mc = row['Avg_MC']
    c2f = (1.8 * temp) + 32 
    if temp > 15.6 and \
       temp <= 49 and \
       mc <= 19:
        df.loc[index, 'Mt'] = 32.3 * (np.exp(
            -4.86 * ( c2f / 60 ) 
            ))

    elif temp > 15.6 and \
         temp <= 26.7 and \
         mc > 19 and mc <= 28:
        df.loc[index, 'Mt'] = np.exp(-0.00493277 + 
            (0.05*(c2f)-3)*(np.log(0.0795012 + 0.0123150*mc))) 

    elif temp > 15.6 and \
         temp <= 26.7 and \
         mc > 28:
        df.loc[index, 'Mt'] = np.exp(2.56683-
            0.0428628 * (1.8 * temp + 32))  
               
    elif temp > 26.7 and \
         temp <= 49 and \
         mc > 19 and mc <= 28:
        df.loc[index, 'Mt'] = 32.3 * (np.exp(-3.48 * 
            c2f)/60) + (mc-19/100) * (np.exp(0.61*(1.8 * temp-28)/60)) 

    elif temp > 26.7 and \
         temp <= 49 and \
         mc > 28:
        df.loc[index, 'Mt'] = 32.3 * (np.exp(-3.48 * 
            c2f)/60) + (0.09 * (np.exp(0.61*(1.8 * temp-28)/60)))
             

    elif temp >= 0 and \
         temp <= 15.6:
        df.loc[index, 'Mt'] = 128.389 * (np.exp(-4.86 * 
            c2f/60))             

In [7]:
df['Mm'] = np.nan

In [8]:
for index, row in df.iterrows():
    temp = row['Avg_Temp']
    mc = row['Avg_MC']
    if mc >= 13 and \
       mc <= 35: 
        df.loc[index, 'Mm'] = 0.103 * (np.exp(
            455/((100 * mc) / (100-mc))**1.53)-
            ((0.845*mc) / (100-mc)) + 1.558)


In [9]:
df['Md'] = np.nan

In [10]:
for index, row in df.iterrows():
    temp = row['Avg_Temp']
    mc = row['Avg_MC']
    D = 0.05
    df.loc[index, 'Md'] = 2.08 * (np.exp(-0.0239 * D)) 
             

In [11]:
df['Mh'] = 1

In [12]:
df['Mf'] = 1

In [13]:
df['m'] = df['Mt'] * df['Mm'] * df['Md']

In [14]:
df['gCO2/kg DM'] = np.nan
df['gCO2/kg DM'] = 1.3 * (np.exp(0.006 * 1/ df['m']) - 1) + (0.015 * 1/ df['m'])

In [15]:
df

,Unnamed: 0,Steady date,OPI_DATE,Avg_Temp,RH,Avg_MC,Mt,Mm,Md,Mh,Mf,m,gCO2/kg DM
0,1,2020-11-03 14:51:00,2020-11-03 14:51:00.000,24,55.64,21,0.436463,2.224319,2.077516,1,1,2.016923,0.011310
1,2,2020-11-03 15:51:00,2020-11-03 15:51:00.000,24,56.38,16,0.073085,15.587184,2.077516,1,1,2.366682,0.009638
2,3,2020-11-03 16:51:00,2020-11-03 16:51:00.000,30,59.72,21,27.106322,2.224319,2.077516,1,1,125.259869,0.000182
3,4,2020-11-03 17:51:00,2020-11-04 16:50:59.995,30,59.72,16,0.030472,15.587184,2.077516,1,1,0.986777,0.023130


In [20]:
#df['mgCO2'] = np.nan
#df['mgCO2'] = df['gCO2/kg DM'] * 1000
#create a new colum
#this new code is what is right from moog
df['gCO2'] = np.nan
df['gCO2'] = df['gCO2/kg DM'] *1000

In [16]:
#df['mgCO2'] = np.nan
#df['mgCO2'] = df['gCO2/kg DM'] * 1000
#create a new colum
#multiply the column by a factor which is the 1kg divided by weight of corn DM
df['gCO2'] = np.nan
df['gCO2'] = df['gCO2/kg DM'] / (1/7679)

In [21]:
#convert to milligram CO2 by multiplying by 1000
df['mgCO2'] = np.nan
df['mgCO2'] = df['gCO2'] * 1000

In [23]:
df['mg/m^3'] = np.nan
df['mg/m^3'] = df['mgCO2'] / 18.76250227

In [24]:
df['CO2 in ppm'] = np.nan
df['CO2 in ppm'] = df['mg/m^3'] * 1.8
df

,Unnamed: 0,Steady date,OPI_DATE,Avg_Temp,RH,Avg_MC,Mt,Mm,Md,Mh,Mf,m,gCO2/kg DM,gCO2,mgCO2,mg/m^3,CO2 in ppm
0,1,2020-11-03 14:51:00,2020-11-03 14:51:00.000,24,55.64,21,0.436463,2.224319,2.077516,1,1,2.016923,0.011310,11.310108,11310.108428,602.803841,1085.046913
1,2,2020-11-03 15:51:00,2020-11-03 15:51:00.000,24,56.38,16,0.073085,15.587184,2.077516,1,1,2.366682,0.009638,9.637920,9637.919667,513.679867,924.623760
2,3,2020-11-03 16:51:00,2020-11-03 16:51:00.000,30,59.72,21,27.106322,2.224319,2.077516,1,1,125.259869,0.000182,0.182023,182.023078,9.701429,17.462572
3,4,2020-11-03 17:51:00,2020-11-04 16:50:59.995,30,59.72,16,0.030472,15.587184,2.077516,1,1,0.986777,0.023130,23.129596,23129.595679,1232.756449,2218.961609


In [19]:
# df['Cumulative CO2 in ppm'] = np.nan
# df['Cumulative CO2 in ppm'] = df['CO2 in ppm'] + 35.968

In [20]:
# df['Amb CO2 + Cumulative CO2 in ppm'] = np.nan
# df['Amb CO2 + Cumulative CO2 in ppmm'] = df['Cumulative CO2 in ppm'] + 420

In [21]:
df.to_csv(path / 'Bin13_steele_cal_update_110121.csv')

In [22]:

# Estimation of Mt (temperature multiplier)
# M is moisture and temp is temperature
# for 15.6 < Temp <= 49C and M <= 19%
#     Mt= 32.3*(e**(-4.86*((1.8*Temp)+32)/60))    ####128.389*(EXP(-4.86*((1.8*G2)+32)/60))
# for 15.6 < Temp <= 26.7C and 19 < M <= 28% 
#     Mt = e**(-0.00493277 + (0.05(1.8*Temp +32)-3)*(ln(0.0795012 + 0.0123150*M)) 
# for 15.6 < Temp <= 26.7C and M > 28%     
#     Mt = e**2.56683-0.0428628*(1.8*Temp + 32)
# for 26.7 < Temp <= 49C and 19 < M <= 28%     
#     Mt= 32.3*(e**-3.48(1.8*Temp + 32)/60) + (M-19/100)*e**0.61(1.8*Temp-28)/60 
# for 26.7 < Temp <= 49C and M > 28%     
#     Mt= 32.3*(e**-3.48(1.8*Temp + 32)/60) + 0.09*e**0.61(1.8*Temp-28)/60 
# for 0 <= Temp <= 15.6C     
#     Mt= 128.389*(e**-4.86(1.8*Temp + 32)/60) 



In [23]:
#   Estimation of Mm (moisture multiplier)
#   for 13 <= M <=35%
#       Mm = 0.103*e**(455/((100*M)/(100-M))**1.53)-((0.845*M)/(100-M))+1.558) ####0.103*(EXP(455/((100*C2)/(100-C2))^1.53)-((0.845*C2)/(100-C2))+1.558)


#   Estimation of Md (Damage multiplier)
#   for 2 <= D <= 41%
#       Md= 2.08*e**(-0.0239*0.05)

#   Estimation of Mh (Genetic Hybrid Multiplier)
#       Mh = 1   ###for generic hybrid

#   estimation of Mf (fungicide multiplier)
#       Mf = 1    ###for no fungicide application

In [24]:
#   Combined multiplier, m
#       m = Mm * Mt * Md * Mh * Mf
#        t = 1      #hourly
#   CO2 prediction, Y in gCO2/kg DM
#        Y =1.3*e**(0.006*t/m)-1)+(0.015*t/m)  ####1.3*(EXP(0.006*A2/R2)-1)+(0.015*A2/R2)
#``````````````````````````````````````````
#   Y in mgCO2      ### though its gCO2 by steele
#        A = Y*1000
#   Y in mg/m3
#       B = A/3.5255    ### 3.5255 M3 reps the vol of air in the corn mass 
#   Y in mgCO2/kg
#   Y in mgCO2/kg